In [1]:
import os, sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

In [2]:
DATA_DIR = '../data'
# mode = 'sample_100'
mode = '1_to_1'
is_cont = 'continuous'

In [3]:
states_results = pd.read_csv(os.path.join(DATA_DIR, f'state_results_{mode}_{is_cont}.csv'))

In [4]:
# states_results.loc[states_results['state_code'] == 'PA','winner'] = 'trump'
# states_results.loc[states_results['state_code'] == 'CA','winner'] = 'trump'

In [5]:
num_vote_to_win = 270
trump_vote = np.sum(states_results[states_results['winner'] == 'trump']['e_vote'])
biden_vote = np.sum(states_results[states_results['winner'] == 'biden']['e_vote'])
print(trump_vote, biden_vote)

121 208


In [6]:
contentious_states = states_results.loc[states_results['winner'] == 'contentious',['state_code', 'e_vote']]
contentious_states

,state_code,e_vote
4,AZ,11
5,NV,6
7,NH,4
9,NJ,14
10,CO,9
11,NM,5
15,NC,15
18,FL,29
19,OH,18
33,TN,11


In [7]:
def predict_winner(states, win_lose):
    trump_cont_vote = trump_vote + np.sum(states.iloc[np.where(np.array(win_lose) == 1)[0]]['e_vote'])
    biden_cont_vote = biden_vote + np.sum(states.iloc[np.where(np.array(win_lose) == 0)[0]]['e_vote'])
    
#     print(trump_cont_vote, biden_cont_vote)
    
    if trump_cont_vote >= num_vote_to_win:
        return 1
    elif biden_cont_vote >= num_vote_to_win:
        return 0
    else:
        return None

In [8]:
get_bin = lambda x, n: format(x, 'b').zfill(n)

In [9]:
total = 0
n_bits = len(contentious_states)
n_possible_scenarios = 2**n_bits
draw = 0
for i in range(n_possible_scenarios):
    win_lose = list(map(int, get_bin(i, n_bits)))
    
    winner = predict_winner(contentious_states, win_lose)
    if winner != None:
        total += winner
    else:
        draw += 1

In [10]:
trump_chance_to_win = total/(n_possible_scenarios)
draw_chance = draw/n_possible_scenarios
biden_chance_to_win = 1 - draw_chance - trump_chance_to_win

In [11]:
print(trump_chance_to_win*100, biden_chance_to_win*100, draw_chance*100)

8.8287353515625 90.6768798828125 0.494384765625


In [12]:
win_chance_dict = dict({'candidate': ['trump', 'biden', 'draw'],
                        'chance_to_win': [f'{trump_chance_to_win*100:.5f}',
                                          f'{biden_chance_to_win*100:.5f}',
                                          f'{draw_chance*100:.5f}']
                       })
win_chance_dict

{'candidate': ['trump', 'biden', 'draw'],
 'chance_to_win': ['8.82874', '90.67688', '0.49438']}

In [13]:
win_chance_df = pd.DataFrame(win_chance_dict)
win_chance_df.to_csv(os.path.join(DATA_DIR, f'win_chance_state_results_{mode}_{is_cont}.csv'), index=False, header=True)